In [1]:
from ACDC.random_walk_classifier import * 
from ACDC.cell_type_annotation import *

In [2]:
import pandas as pd
import numpy as np
from collections import Counter

path = 'D:/research/Liza/cytof/CyTOF existing methods and challenges/programs/autogating/acdc/data/'
df = pd.read_csv(path + 'data2.csv', sep=',', header = 0)

df = df.drop(['id'], axis = 1)

channels = [item for item in df.columns[:-1]]
df.columns = channels + ['cell_type']

#df = df.loc[df['cell_type'] != 'NotDebrisSinglets']

table = pd.read_csv(path + 'data_table2.csv', sep=',', header=0, index_col=0)
table = table.fillna(0)

cts, channels = get_label(table)

X0= np.arcsinh((df[channels].values - 1.0)/5.0)

In [3]:
idx2ct = [key for idx, key in enumerate(table.index)]
idx2ct.append('unknown')

ct2idx = {key:idx for idx, key in enumerate(table.index)}
ct2idx['unknown'] = len(table.index)
        
ct_score = np.abs(table.as_matrix()).sum(axis = 1)

## compute manual gated label
y0 = np.zeros(df.cell_type.shape)

for i, ct in enumerate(df.cell_type):
    if ct in ct2idx:
        y0[i] = ct2idx[ct]
    else:
        y0[i] = ct2idx['unknown']

C:\Users\liupeng2117\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [4]:
from sklearn.metrics import accuracy_score, confusion_matrix
import phenograph
from sklearn.model_selection import StratifiedKFold
import pickle
n_neighbor = 10
thres = 0.5

In [6]:
import time

X1 = np.arange(0,len(X0),1)
#skf0 = StratifiedKFold(n_splits=5, shuffle=True, random_state=None)
#skf=skf0.split(X1, y0)
#print(skf)  

result = []
score_final = []

process_time = []
c = 0

#for tr, te in skf:
#print('%02d th batch' % c)
    #if c == 1:
    #    break
    #c += 1
    
X = X0.copy()
y_true = y0.copy()

    #X = X[tr, :]
    #y_true = y_true[tr]

mk_model =  compute_marker_model(pd.DataFrame(X, columns = channels), table, 0.0)

    ## compute posterior probs
tic = time.clock()
score = get_score_mat(X, [], table, [], mk_model)
score = np.concatenate([score, 1.0 - score.max(axis = 1)[:, np.newaxis]], axis = 1)    
    
    ## get indices     
ct_index = get_unique_index(X, score, table, thres)
    
    ## baseline - classify events    
y_pred_index = np.argmax(score, axis = 1)
    
toc = time.clock()
time0 = toc - tic
    
    
    
    ## running ACDC
tic = time.clock()
res_c = get_landmarks(X, score, ct_index, idx2ct, phenograph, thres)

landmark_mat, landmark_label = output_feature_matrix(res_c, [idx2ct[i] for i in range(len(idx2ct))]) 

landmark_label = np.array(landmark_label)

lp, y_pred = rm_classify(X, landmark_mat, landmark_label, n_neighbor)

    #process_time.append(toc-tic)
    
    #res = phenograph.cluster(X, k=30, directed=False, prune=False, min_cluster_size=10, jaccard=True,
    #                    primary_metric='euclidean', n_jobs=-1, q_tol=1e-3)
    
toc = time.clock()
time1 = toc - tic
    
    
    ## running phenograph classification
    #tic = time.clock()
    #y_pred_oracle = np.zeros_like(y_true)
    #for i in range(max(res[0])+1):
    #    ic, nc = Counter(y_true[res[0] == i]).most_common(1)[0]
    #    y_pred_oracle[res[0] == i] = ic
        
score_final.append([accuracy_score(y_true, [ct2idx[c] for c in y_pred]), 
                    accuracy_score(y_true, y_pred_index)])
    
    #toc = time.clock()
    #time2 = toc - tic   
    
    
result.append((y_true, y_pred, y_pred_index))
process_time.append((time0, time1))
    
    #pickle.dump(result, open('processed_file/AML/event_classidication_AML.p', 'wb'))

C:\Users\liupeng2117\Anaconda3\lib\ACDC\cell_type_annotation.py:32: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  tmp = df[mk].as_matrix()
C:\Users\liupeng2117\Anaconda3\lib\ACDC\cell_type_annotation.py:32: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  tmp = df[mk].as_matrix()
C:\Users\liupeng2117\Anaconda3\lib\ACDC\cell_type_annotation.py:32: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  tmp = df[mk].as_matrix()
C:\Users\liupeng2117\Anaconda3\lib\ACDC\cell_type_annotation.py:32: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  tmp = df[mk].as_matrix()
C:\Users\liupeng2117\Anaconda3\lib\ACDC\cell_type_annotation.py:32: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  tmp = df[mk].as_matrix()
C:\Users\liupeng2117\Anaconda3\lib\ACDC\cell_type_annotation.py:32: Fu

Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm


C:\Users\liupeng2117\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\liupeng2117\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


Neighbors computed in 1.9828155040740967 seconds
Jaccard graph constructed in 3.718597412109375 seconds
Wrote graph to binary file in 1.0351593494415283 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.669268
After 9 runs, maximum modularity is Q = 0.670613
After 13 runs, maximum modularity is Q = 0.673076
Louvain completed 33 runs in 15.455009698867798 seconds
PhenoGraph complete in 22.241498708724976 seconds
Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 0.1070561408996582 seconds
Jaccard graph constructed in 2.2567930221557617 seconds
Wrote graph to binary file in 0.1356339454650879 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.650369
After 3 runs, maximum modularity is Q = 0.652887
Louvain completed 23 runs in 0.905219316482544 seconds
PhenoGraph complete in 3.412646770477295 seconds
Finding 30 nearest neighbors using minkowski metric and 'auto' algor

C:\Users\liupeng2117\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [14]:
print(process_time)
resultdf=pd.DataFrame(y_pred,y_true, ['predicted'])
resultdf.to_csv("D:/research/Liza/cytof/CyTOF existing methods and challenges/programs/autogating/acdc/result.csv")

[(1.7662609000000202, 1993.2490729999997), (1.7662609000000202, 1993.2490729999997)]


In [10]:
np.mean(score_final, axis = 0)

array([0.75794732, 0.62254552])

In [18]:
result2=np.array(result,dtype=)
result2=result2[0][0]

In [20]:
np.savetxt("result.csv",result2)

TypeError: Mismatch between array dtype ('<U32') and format specifier ('%.18e')